# DataFrame

Credits: data taken from [retail data](https://github.com/databricks/Spark-The-Definitive-Guide/tree/master/data/retail-data) example from Spark: The Definitive Guide Orielly Publishers, Zaharia et al

* Most famous way of interacting with Spark is through DataFrame
* Faster than RDD as it has an optimization layer that converts the user request to optimized plan for execution 
* 
* ![img](img/RDD_VS_DF.png)
* Credits: [Image taken from Databricks website](https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html)
* Using RDDs require knowledge of atleast one functional language
* DataFrame API executes at a language neutral level so has similar execution times in both python and scala
* 
* ![img](img/SQL_OPT.png)
* Credits: [Image taken from Databricks website](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)
* Also comes with JDBC and ODBC connections which enable BI tool connections

In [ ]:
from pyspark.sql import Row
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pylab as P
from pyspark.sql.types import *

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("ImportCSV").getOrCreate()

In [ ]:
static = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/home/jovyan/dayOne.csv")

In [ ]:
static.createOrReplaceTempView("retail_Data")

In [ ]:
static.printSchema()

In [ ]:
spark.sql('SELECT * FROM retail_Data ORDER BY UnitPrice DESC').show(5)

In [ ]:
from pyspark.sql.functions import window, column, desc, col
static.selectExpr("CustomerID","(UnitPrice * Quantity) as total_cost").groupBy(col("CustomerID")).sum("total_cost").show(5)

In [ ]:
spark.sql('SELECT CustomerID, SUM(UnitPrice*Quantity) as total_cost FROM retail_Data Group BY CustomerID').show(5)

In [ ]:
spark.sql('SELECT CustomerID, SUM(UnitPrice*Quantity) as total_cost FROM retail_Data Group BY CustomerID').explain()

In [ ]:
foo = spark.sql('SELECT * FROM retail_Data ORDER BY UnitPrice DESC')

In [ ]:
test = foo.where("Country = 'United Kingdom'")

In [ ]:
train = foo.where("Country != 'United Kingdom'")

In [ ]:
test.count()

In [ ]:
train.count()

In [ ]:
UP = spark.sql('SELECT UnitPrice FROM retail_Data')

In [ ]:
UPList = UP.rdd.map(lambda p: p.UnitPrice).collect()

In [ ]:
UP.describe().show()

In [ ]:
plt.hist(UPList)

## Write CSV

In [ ]:
UK = spark.sql("SELECT * FROM retail_Data WHERE Country = 'United Kingdom'")

In [ ]:
UK.repartition(1).write.csv("UK.csv", sep=',', header=True)

### like CSV you can also import JSON

### You can convert a dataframe into a rdd by using the .rdd notation

In [ ]:
UK.rdd.count()

In [ ]:
UK.rdd.take(1)

## DropMalformed

In [ ]:
df = spark.read.format("csv").option("header", True).option("mode", "DROPMALFORMED").option("maxMalformedLogPerPartition", 128).load('/home/jovyan/airtravel.csv')

In [ ]:
df.show(5)

In [ ]:
df2 = spark.read.format("csv").option("header", True).option("mode", "DROPMALFORMED").option("maxMalformedLogPerPartition", 128).load('/home/jovyan/airtravel_2020.csv')

In [ ]:
df2.show(5)

In [ ]:
df.createOrReplaceTempView("Pre")
df2.createOrReplaceTempView("Post")

In [ ]:
spark.sql('SELECT * FROM Pre as P1 JOIN Post as P2 ON P1.MonthID = P2.MonthID').show(5)

In [ ]:
df_full = df.join(df2, ['MonthId'])

In [ ]:
df_full.show()